In [1]:
import time
import numpy as np
from dataclasses import dataclass, field
import zipfile
import xml.etree.ElementTree as ET
import os

In [2]:
# --- Part 0: Dynamic SysML Model Loader ---
# This section provides the functionality to parse the .gaphor model file
# and load parameters directly from the system design. This makes the SysML
# model the "single source of truth" for our analysis.

def load_params_from_sysml(model_path: str) -> dict:
    """
    Parses a .gaphor file to extract default values from SysML blocks.

    Args:
        model_path (str): The full path to the .gaphor model file.

    Returns:
        dict: A dictionary containing the loaded parameters.
    """
    params = {}
    
    if not os.path.exists(model_path):
        print(f"Warning: SysML model file not found at '{model_path}'. Using hardcoded defaults.")
        return params

    print(f"\n--- Loading parameters from SysML model: {model_path} ---")
    
    try:
        # A .gaphor file is a zip archive containing model.xml
        with zipfile.ZipFile(model_path, 'r') as z:
            with z.open('model.xml') as model_file:
                tree = ET.parse(model_file)
                root = tree.getroot()

                # Find all Blocks (which are a type of Class in the XML)
                # We need to find the correct XML tags by inspecting the model.xml
                ns = {'UML': 'http://www.omg.org/spec/UML/20110701'} # Namespace for UML
                
                # Find the Satellite block's properties
                satellite_block = root.find(".//UML:Class[@name='Satellite']", ns)
                if satellite_block:
                    for prop in satellite_block.findall('ownedAttribute'):
                        name = prop.get('name')
                        default_value_element = prop.find('defaultValue')
                        if default_value_element is not None:
                            value = float(default_value_element.get('value'))
                            params[f'satellite_{name}'] = value
                            print(f"    Loaded Satellite.{name}: {value}")

                # Find the OrbitalPlane block's properties
                plane_block = root.find(".//UML:Class[@name='Orbital Plane']", ns)
                if plane_block:
                     for prop in plane_block.findall('ownedAttribute'):
                        name = prop.get('name')
                        default_value_element = prop.find('defaultValue')
                        if default_value_element is not None:
                            value = float(default_value_element.get('value'))
                            params[f'plane_{name}'] = value
                            print(f"    Loaded Orbital Plane.{name}: {value}")
    except Exception as e:
        print(f"Error parsing SysML model: {e}. Using hardcoded defaults.")

    return params

In [3]:
#--- Part 1: SysML Model Representation ---
# The defaults here will be used if the SysML model can't be loaded.
@dataclass
class Satellite:
    """Represents the Satellite block from the SysML model."""
    cost_per_sat: float = 500000  # Cost in USD for one satellite
    mass_per_sat: float = 250     # Mass in kg for one satellite

@dataclass
class OrbitalPlane:
    """Represents the OrbitalPlane block from the SysML model."""
    altitude: float = 700.0   # Altitude in km
    inclination: float = 55.0 # Inclination in degrees
    sats_in_plane: int = 6

@dataclass
class Constellation:
    """Represents the Constellation block, aggregating other components."""
    name: str = "QEC-DEO Constellation"
    num_planes: int = 4
    total_satellites: int = field(init=False)
    total_cost: float = field(init=False, default=0.0)
    mean_revisit_time: float = field(init=False, default=float('inf'))
    satellite_params: Satellite = field(default_factory=Satellite)
    plane_params: list[OrbitalPlane] = field(default_factory=list)

    def __post_init__(self):
        """Calculates derived properties after the object is created."""
        if not self.plane_params:
            self.plane_params = [OrbitalPlane() for _ in range(self.num_planes)]
        self.total_satellites = sum(p.sats_in_plane for p in self.plane_params)

In [4]:
# --- Part 2: GMAT and Cost Model Simulation ---
def run_gmat_simulation(constellation: Constellation) -> float:
    """Placeholder function to simulate running a GMAT analysis."""
    print(f"\n--- Simulating GMAT for {constellation.total_satellites} satellites ---")
    print(f"    Planes: {constellation.num_planes}, Altitude: {constellation.plane_params[0].altitude} km, Inclination: {constellation.plane_params[0].inclination} deg")
    time.sleep(0.1)
    base_revisit = 24.0
    satellite_factor = (constellation.total_satellites / 4.0)**(-0.9)
    altitude_factor = 1 / (1 + (constellation.plane_params[0].altitude - 500) / 1000)
    inclination_penalty = 1 + (abs(constellation.plane_params[0].inclination - 45) / 90)**2
    revisit_time = base_revisit * satellite_factor * altitude_factor * inclination_penalty
    print(f"--- GMAT Result: Mean Revisit Time = {revisit_time:.2f} hours ---")
    return revisit_time

def calculate_total_cost(constellation: Constellation) -> float:
    """Calculates the total constellation cost based on the SysML model parameters."""
    cost_per_kg_to_orbit = 8000
    manufacturing_cost = constellation.total_satellites * constellation.satellite_params.cost_per_sat
    launch_cost = constellation.total_satellites * constellation.satellite_params.mass_per_sat * cost_per_kg_to_orbit
    total_cost = manufacturing_cost + launch_cost
    print(f"--- Cost Model: Total Estimated Cost = ${total_cost:,.2f} ---")
    return total_cost


In [5]:
# --- Part 3: Objective Function ---
def objective_function(num_planes: float, sats_per_plane: float, altitude: float, inclination: float, base_satellite: Satellite) -> float:
    """The main objective function for the optimizer."""
    num_planes_int = int(round(num_planes))
    sats_per_plane_int = int(round(sats_per_plane))
    planes = [OrbitalPlane(altitude=altitude, inclination=inclination, sats_in_plane=sats_per_plane_int) for _ in range(num_planes_int)]
    constellation = Constellation(num_planes=num_planes_int, plane_params=planes, satellite_params=base_satellite)
    revisit_time = run_gmat_simulation(constellation)
    total_cost = calculate_total_cost(constellation)
    constellation.mean_revisit_time = revisit_time
    constellation.total_cost = total_cost
    cost_score = total_cost / 35_000_000
    revisit_score = revisit_time / 3.0
    penalty = 0
    if revisit_time > 4.0:
        penalty += (revisit_time - 4.0) * 10
    if total_cost > 50_000_000:
        penalty += (total_cost - 50_000_000) / 1_000_000
    w1, w2 = 0.6, 0.4
    final_score = (w1 * cost_score) + (w2 * revisit_score) + penalty
    print(f"===> Objective Score: {final_score:.4f} (Cost: {cost_score:.2f}, Revisit: {revisit_score:.2f}, Penalty: {penalty:.2f})")
    return final_score


In [6]:
# --- Part 4: Main Execution Block ---
if __name__ == "__main__":
    print("====== Starting Constellation Optimization Backend ======")

    # --- DYNAMICALLY LOAD FROM SYSML ---
    # Replace 'path/to/your/model.gaphor' with the actual path to your saved Gaphor file
    sysml_model_path = '/home/michael/Documents/Sat_proj/qec_deo_model.gaphor' 
    loaded_params = load_params_from_sysml(sysml_model_path)
    
    # Create a base Satellite object, using loaded SysML values or defaults
    base_satellite_template = Satellite(
        cost_per_sat=loaded_params.get('satellite_costPerSat', 500000),
        mass_per_sat=loaded_params.get('satellite_massPerSat', 250)
    )
    print(f"\nUsing base satellite parameters: {base_satellite_template}")
    
    bounds = [(2, 10), (3, 12), (550, 1000), (30, 90)]

    print("\n--- Testing with a baseline design ---")
    # Note: We now pass the loaded satellite template to the objective function
    baseline_score = objective_function(
        num_planes=4, 
        sats_per_plane=6, 
        altitude=700, 
        inclination=55,
        base_satellite=base_satellite_template
    )
    
    print("\n--- Testing with another sample design ---")
    sample_score = objective_function(
        num_planes=8, 
        sats_per_plane=10, 
        altitude=850, 
        inclination=45,
        base_satellite=base_satellite_template
    )

    print("\n====== Backend Test Complete ======")
    print(f"\nNext step: Feed the 'objective_function' into a classical or quantum optimizer.")

====== Starting Constellation Optimization Backend ======

--- Loading parameters from SysML model: /home/michael/Documents/Sat_proj/qec_deo_model.gaphor ---
Error parsing SysML model: File is not a zip file. Using hardcoded defaults.

Using base satellite parameters: Satellite(cost_per_sat=500000, mass_per_sat=250)

--- Testing with a baseline design ---

--- Simulating GMAT for 24 satellites ---
    Planes: 4, Altitude: 700 km, Inclination: 55 deg
--- GMAT Result: Mean Revisit Time = 4.04 hours ---
--- Cost Model: Total Estimated Cost = $60,000,000.00 ---
===> Objective Score: 11.9334 (Cost: 1.71, Revisit: 1.35, Penalty: 10.37)

--- Testing with another sample design ---

--- Simulating GMAT for 80 satellites ---
    Planes: 8, Altitude: 850 km, Inclination: 45 deg
--- GMAT Result: Mean Revisit Time = 1.20 hours ---
--- Cost Model: Total Estimated Cost = $200,000,000.00 ---
===> Objective Score: 153.5885 (Cost: 5.71, Revisit: 0.40, Penalty: 150.00)

====== Backend Test Complete =====